<img src = "./resources/images/header_banner_3.jpeg" alt = "Encabezado MLDS" width = "100%">  </img>

# **Preparación de los datos**
---

## **0. Integrantes del equipo de trabajo**
---

<table><thead>
  <tr>
    <th>#</th>
    <th>Integrante</th>
    <th>Documento de identidad</th>
  </tr></thead>
<tbody>
  <tr>
    <td>1</td>
    <td>Laura Alejandra Díaz López</td>
    <td>1010018062</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Diego Alejandro Feliciano Ramos</td>
    <td>1024586904</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Geraldine Gracia Ruiz</td>
    <td>1032488268</td>
  </tr>
</tbody>
</table>

## **1. Limpieza de los Datos**
---

La elección de las técnicas de preprocesamiento puede diferir en cada conjunto de datos. Recuerde que es posible aplicar, según sea necesario (no necesariamente todas), las técnicas generales que se han explorado en el curso. La elección dependerá del tipo de datos con el que esté trabajando.

A lo largo de esta entrega, busque responder las siguientes preguntas:

- ¿Cuáles fueron los criterios utilizados para identificar y tratar valores atípicos, datos faltantes o cualquier otra anomalía en el conjunto de datos durante el proceso de limpieza?
- ¿Cómo se justificaría la necesidad de cada paso de preprocesamiento en términos de mejora de la calidad de los datos y preparación para el análisis subsiguiente?

A continuación encontrará los puntos a tratar a medida que va realizando la preparación de los datos. En cada punto defina el estado en que se encontraba el dataset, ademas de explicar y justificar las acciones y decisiones que se tomaron.

In [ ]:
# Instalamos los paquetes necesarios para el notebook en caso de trabajar desde Google Colab
!pip install wget pandas ydata-profiling ipywidgets

In [ ]:
import wget, pandas as pd

# Descargar el dataset transformado de la entrega anterior del repositorio publico
url = 'https://github.com/MLDS-UN-ProjectTeam/final-term-project/raw/main/resources/data/Saber_11_2023-2_-_Transformado.csv?download='
downloaded_file_name = 'Saber_11_2023-2_-_Transformado.csv'
wget.download(url = url, out = downloaded_file_name)

# Cargar en el notebook el conjunto de datos con las restricciones conocidas:
# - Delimitador: ¬ 
# - Quoting = 3 => Sin comillas para todas las variables
saber11_dataframe = pd.read_csv(downloaded_file_name, delimiter=',', quoting=3)

### **1.1. Valores faltantes**
---
Al encontrarnos con valores faltantes en el conjunto de datos, es crucial preguntarse:
1. **¿Cómo afectan estos valores a la integridad y representatividad de la información?**

La ausencia o falta de datos en un conjunto puede impactar negativamente tanto la calidad como la precisión de la información analizada. Esto ocurre de dos maneras principales:
* En cuanto a la calidad de los datos:
    * Los análisis pueden ser imprecisos debido a cálculos erróneos.
    * Se puede perder información valiosa, limitando la utilidad del análisis.
    * Los modelos predictivos pueden fallar o funcionar de manera deficiente.
* Respecto a la precisión de los datos:
    * Pueden surgir sesgos si la falta de datos no es aleatoria.
    * Por ejemplo, en estudios médicos, si los pacientes más graves tienen más datos faltantes, se podría subestimar la severidad de la enfermedad.
    * La capacidad de detectar patrones significativos se reduce al haber menos datos disponibles.
Para abordar estos problemas, existen varias estrategias:
* Eliminar registros incompletos, aunque esto puede resultar en pérdida de información.
* Estimar los valores faltantes, pero esto puede introducir suposiciones inexactas.
* Utilizar algoritmos diseñados para manejar datos incompletos.

2. **¿Cómo se identificaron los valores faltantes en el conjunto de datos?**

In [41]:
#Para la identificación de valores faltantes usamos pandas y varias de las funciones que nos ofrece su api, aquí será descrito cada método que se usó y la finalidad con la que se usó

# Contar el total de valores faltantes en el DataFrame
total_faltantes = saber11_dataframe.isnull().sum().sum()
print('\nTotal faltantes = ' + str(total_faltantes))

#Cantidad de datos nulos por columna
columnas_con_faltantes = saber11_dataframe.isnull().sum()
columnas_con_faltantes = columnas_con_faltantes[columnas_con_faltantes != 0]
print(columnas_con_faltantes)

# Rellenamos columnas de interés con la media de las


Total faltantes = 2386169
ESTU_GENERO                           7
ESTU_PRESENTACIONSABADO             419
ESTU_LENGUANATIVA                541839
FAMI_ESTRATOVIVIENDA              49105
FAMI_PERSONASHOGAR                29154
FAMI_CUARTOSHOGAR                 30336
FAMI_EDUCACIONPADRE               43179
FAMI_EDUCACIONMADRE               43450
FAMI_TRABAJOLABORPADRE            30611
FAMI_TRABAJOLABORMADRE            30306
FAMI_TIENEINTERNET                43908
FAMI_TIENECOMPUTADOR              30146
FAMI_TIENELAVADORA                30144
FAMI_TIENEHORNOMICROOGAS          30747
FAMI_TIENESERVICIOTV              45184
FAMI_TIENEAUTOMOVIL               31190
FAMI_TIENEMOTOCICLETA             30623
FAMI_TIENECONSOLAVIDEOJUEGOS      31397
FAMI_NUMLIBROS                    48493
FAMI_COMELECHEDERIVADOS           50599
FAMI_COMECARNEPESCADOHUEVO        45826
FAMI_COMECEREALFRUTOSLEGUMBRE     49234
FAMI_SITUACIONECONOMICA           32926
ESTU_DEDICACIONLECTURADIARIA      44904
ESTU_DEDICACI



3. **¿Cuáles fueron los criterios para decidir si rellenar con valores estimados o eliminar los valores faltantes? En caso que aplique, ¿qué método de relleno se utilizó y por qué se consideró apropiado?**


4. **¿Se realizó un análisis de la distribución de los valores faltantes en relación con las variables clave? Por ejemplo, si hay valores faltantes en una variable crítica, ¿cómo podría afectar la interpretación de los resultados?**

### **1.2. Valores duplicados**
----
Ante la presencia de valores duplicados, es esencial reflexionar sobre su impacto en los resultados.

* ¿Cómo se identificaron y manejaron los valores duplicados en el conjunto de datos?
* ¿Hubo alguna consideración especial al tratar duplicados en función de ciertas variables?
* ¿Pueden estos duplicados sesgar el análisis, y en qué medida?
* ¿Se justifica la eliminación de duplicados para mantener la coherencia y singularidad de los datos?
* ¿Cómo se aseguró de que la eliminación de duplicados no afectara negativamente la representación del conjunto de datos?

La identificación y eliminación de duplicados busca asegurar que cada registro aporte información única y valiosa.

---**INGRESE SU RESPUESTA**---

In [ ]:
# ---INGRESE SU CÓDIGO---

### **1.3. Valores atípicos**
---
Al abordar valores atípicos, es relevante cuestionarse sobre la naturaleza de estos puntos extremos.
* ¿Son errores de medición o representan información válida pero excepcional?
* ¿Qué criterios o técnicas se utilizaron para identificar los valores atípicos?
* ¿Se aplicaron métodos estadísticos o visuales para detectar los valores atípicos?
* ¿Cuál fue la decisión final sobre cómo tratar los valores atípicos y por qué?

La elección entre eliminar o transformar estos valores debe basarse en su influencia en la distribución y la interpretación de los resultados, asegurando que el análisis refleje de manera precisa la realidad del conjunto de datos.

---**INGRESE SU RESPUESTA**---

In [ ]:
# ---INGRESE SU CÓDIGO---

### **1.4. Datos Inconsistentes**
---
Frente a inconsistencias en los datos, es importante analizar cómo estas discrepancias podrían afectar la comparabilidad y comprensión de la información.

* ¿Cómo se identificaron las inconsistencias en los datos?
* ¿Las inconsistencias son lo suficientemente significativas como para justificar una estandarización?
* ¿Cómo se pueden homogeneizar unidades y formatos para garantizar coherencia en el análisis?
* ¿Qué estrategias se implementaron para abordar las inconsistencias y asegurar la coherencia?
* ¿Se establecieron reglas de validación específicas para garantizar la consistencia?

---**INGRESE SU RESPUESTA**---

In [ ]:
# ---INGRESE SU CÓDIGO---

### **1.5. Datos corruptos**
---
En la detección de datos corruptos, pregunte si hay posibles errores de entrada o problemas de transferencia que podrían haber afectado la integridad de los datos.
* ¿Cómo se determinó la presencia de datos corruptos en el conjunto de datos?
* ¿Qué medidas se tomaron para corregir o eliminar los datos corruptos?
* ¿Hubo casos en los que la corrupción de datos no pudo ser manejada y cómo se justificó esa decisión?

---**INGRESE SU RESPUESTA**---

In [ ]:
# ---INGRESE SU CÓDIGO---

### **1.6. Selección de datos**
---
En la selección de datos, reflexione sobre las variables que son realmente relevantes para sus objetivos de análisis.
* ¿Qué criterios se utilizaron para seleccionar los datos relevantes para el análisis?
* ¿Se aplicaron técnicas de muestreo o filtrado para reducir el tamaño del conjunto de datos?
* ¿Cómo se justificó la inclusión o exclusión de ciertas variables en la selección de datos?

---**INGRESE SU RESPUESTA**---

In [ ]:
# ---INGRESE SU CÓDIGO---

## **Créditos**
* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).
    
**Universidad Nacional de Colombia** - *Facultad de Ingeniería*